In [ ]:
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
image = Image.open('../data/scenes/sfbay_1.png')
image

In [ ]:
orig_width = image.size[0]
orig_height = image.size[1]

In [ ]:
HEIGHT = 80
WIDTH = 80
DEPTH = 3
STEP = 10

In [ ]:
toy = image.copy()

## Cropping images

In [ ]:
toy.crop((0, 0, 80, 400))

In [ ]:
toy.crop((left, upper, right - 80, lower - 80))

## Drawing boxes

In [ ]:
# draw a box
toy = image.copy()
draw = ImageDraw.Draw(toy)
draw.rectangle((0, 0, 80, 80), outline='red', width=5)
toy

In [ ]:
left, upper, right, lower = toy.getbbox()
left, upper, right, lower

In [ ]:
# num boxes by width
STEP = 15
w = (orig_width - (WIDTH - STEP)) // STEP
h = (orig_height - (HEIGHT - STEP)) // STEP
print('Total image chips:', h * w)

In [ ]:
step_sizes = list(range(10, 51))

num_boxes = []
for step in step_sizes:
    w = (orig_width - (WIDTH - step)) // step
    h = (orig_height - (HEIGHT - step)) // step
    num_boxes.append(w * h)
    
plt.plot(step_sizes, num_boxes);

## Draw bounding boxes over every potential image chip

In [ ]:
HEIGHT = 80
WIDTH = 80
DEPTH = 3
STEP = 10

toy = image.copy()
draw = ImageDraw.Draw(toy)

for i in range((orig_height - (WIDTH - STEP)) // STEP):  # rows
    upper = STEP * i
    lower = upper + HEIGHT
    
    for j in range((orig_width - (WIDTH - STEP)) // STEP):
        lefter = j * STEP
        righter = lefter + WIDTH
        
        draw.rectangle((lefter, upper, righter, lower), outline='red', width=2)

#         break
#     break

toy

## Crop image and convert to bytes

In [ ]:
# Find a couple points by trial and error
x, y = 1680, 460

toy = image.copy()
draw = ImageDraw.Draw(toy)
draw.line((0, y) + (toy.size[0], y), fill='red', width=5)
draw.line((x, 0) + (x, toy.size[1]), fill='red', width=5)
toy

In [ ]:
l = 1640
t = 430
toy = image.copy()
toy = toy.crop((l, t, l + WIDTH, t + HEIGHT))
toy = toy.crop((l + 200, t + 200, l + WIDTH + 200, t + WIDTH + 200))
clip1 = toy.copy()
clip1

In [ ]:
# Works!
# toy.tobytes()

## AutoML prediction on an image chip

In [ ]:
from google.cloud import automl_v1beta1 as automl

In [ ]:
# constants
PROJECT = 'reliable-realm-222318'
REGION = 'us-central1'
MODEL_ID = 'ICN5377742494555644521'
IMG_PATH = '../data/imgs/ship/1__20160710_182140_0c78__-122.33185409502408_37.74917343586839.png'
THRESHOLD = '0.5'

In [ ]:
automl_client = automl.AutoMlClient()

model_path = automl_client.model_path(PROJECT, REGION, MODEL_ID)

### Image from file

In [ ]:
prediction_client = automl.PredictionServiceClient()

with open(IMG_PATH, 'rb') as img_file:
    content = img_file.read()
    
payload = {'image': {'image_bytes': content}}

params = {}
if THRESHOLD:
    params = {'score_threshold': THRESHOLD}
    
response = prediction_client.predict(model_path, payload, params)

In [ ]:
response

In [ ]:
type(content)

### Image from crop

In [ ]:
from io import BytesIO

In [ ]:
type(clip1.tobytes())

In [ ]:
image_bytes = io.BytesIO()
clip1.save(image_bytes, format='PNG')

payload = {'image': {'image_bytes': image_bytes.getvalue()}}

params = {}
if THRESHOLD:
    params = {'score_threshold': THRESHOLD}
    
response = prediction_client.predict(model_path, payload, params)

In [ ]:
response

# AAAAGGHGGGGHGGGHHHHHHHHHHH!!!!
![Yes!](https://i.imgflip.com/wnv53.jpg)
# YES!

## Crop scene into clips, make predictions, save all prediction/coordinate sets

In [ ]:
from PIL import Image, ImageDraw
from google.cloud import automl_v1beta1 as automl
from io import BytesIO

In [ ]:
# Image Config
HEIGHT = 80
WIDTH = 80
DEPTH = 3
STEP = 25

# AutoML Config
PROJECT = 'reliable-realm-222318'
REGION = 'us-central1'
MODEL_ID = 'ICN5377742494555644521'
THRESHOLD = '0.5'

In [ ]:
# Load image
image = Image.open('../../data/scenes/sfbay_1.png')
toy = image.copy()

In [ ]:
# yield sequence of 4 coord tuples
# crop and predict on that clip
# use cropping function in prediction function

In [ ]:
def generate_clip_bboxes(image, clip_height, clip_width, step_size):
    '''Create generator of (left, top, right, bottom) 4 coord tuple for
    every potential image clip in the provided image, of size
    clip_height x clip_width
    
    image = PIL image
    
    Returns generator
    '''
    coords = []
    
    # Get original img size
    img_height, img_width = image.size
    
    for i in range((img_height - (clip_height - step_size)) // step_size):
        upper = step_size * i
        lower = upper + clip_height
        
        for j in range((img_width - (clip_width - step_size)) // step_size):
            left = j * step_size
            right = left + clip_width
            
            yield (left, upper, right, lower)

In [ ]:
def predict_clips(bbox_gen, image):
    '''doc'''

    # Setup clients
    automl_client = automl.AutoMlClient()
    prediction_client = automl.PredictionServiceClient()
    model_path = automl_client.model_path(PROJECT, REGION, MODEL_ID)

    predictions = {}
    ship_count = 0
    total_count = 0
    
    for coords in bbox_gen:
        # crop clip
        clip = image.copy()
        clip = clip.crop(coords)
        
        # save to bytes
        image_bytes = BytesIO()
        clip.save(image_bytes, format='PNG')

        payload = {'image': {'image_bytes': image_bytes.getvalue()}}
        response = prediction_client.predict(model_path, payload)
        
        print(response)

        predictions[coords] = response
        
        for pred in response.payload:
            total_count += 1
            if pred.display_name == 'ship' and pred.classification.score > 0.9:
                ship_count += 1
                print('Found a ship!')

        
        input() # ghetto debugger

In [ ]:
bboxes = generate_clip_bboxes(toy, HEIGHT, WIDTH, STEP)
predict_clips(bboxes, toy)

In [ ]:
toy

In [ ]:
## DONT RUN; REFERENCE ONLY

# drawing
for i in range((orig_height - (WIDTH - STEP)) // STEP):  # rows
    upper = STEP * i
    lower = upper + HEIGHT
    
    for j in range((orig_width - (WIDTH - STEP)) // STEP):
        lefter = j * STEP
        righter = lefter + WIDTH
        
        draw.rectangle((lefter, upper, righter, lower), outline='red', width=2)

        
# cropping
l = 1640
t = 430
toy = image.copy()
toy = toy.crop((l, t, l + WIDTH, t + HEIGHT))
clip1 = toy.copy()
clip1

# predicting
image_bytes = io.BytesIO()
clip1.save(image_bytes, format='PNG')

payload = {'image': {'image_bytes': image_bytes.getvalue()}}

params = {}
if THRESHOLD:
    params = {'score_threshold': THRESHOLD}
    
response = prediction_client.predict(model_path, payload, params)

In [ ]:
response

## Predictions with AutoML

In [ ]:
from PIL import Image, ImageDraw
from google.cloud import automl_v1beta1 as automl
from io import BytesIO

# Image Config
HEIGHT = 80
WIDTH = 80
DEPTH = 3
STEP = 25

# AutoML Config
PROJECT = 'reliable-realm-222318'
REGION = 'us-central1'
MODEL_ID = 'ICN5377742494555644521'
THRESHOLD = '0.5'

# Load image
image = Image.open('../../data/scenes/sfbay_1.png')
toy = image.copy()

In [ ]:
def generate_clip_bboxes(image, clip_height, clip_width, step_size):
    '''Create generator of (left, top, right, bottom) 4 coord tuple for
    every potential image clip in the provided image, of size
    clip_height x clip_width
    
    image = PIL image
    
    Returns generator
    '''
    coords = []
    
    # Get original img size
    img_height, img_width = image.size
    
    for i in range((img_height - (clip_height - step_size)) // step_size):
        upper = step_size * i
        lower = upper + clip_height
        
        for j in range((img_width - (clip_width - step_size)) // step_size):
            left = j * step_size
            right = left + clip_width
            
            yield (left, upper, right, lower)

In [ ]:
def predict_clips(bbox_gen, image):
    '''doc'''

    # Setup clients
    automl_client = automl.AutoMlClient()
    prediction_client = automl.PredictionServiceClient()
    model_path = automl_client.model_path(PROJECT, REGION, MODEL_ID)

    predictions = {}
    ship_count = 0
    total_count = 0
    
    for coords in bbox_gen:
        # crop clip
        clip = image.copy()
        clip = clip.crop(coords)
        
        # save to bytes
        image_bytes = BytesIO()
        clip.save(image_bytes, format='PNG')

        payload = {'image': {'image_bytes': image_bytes.getvalue()}}
        response = prediction_client.predict(model_path, payload)
        
        print(response)

        predictions[coords] = response
        
        for pred in response.payload:
            total_count += 1
            if pred.display_name == 'ship' and pred.classification.score > 0.9:
                ship_count += 1
                print('Found a ship!')

        
        input() # ghetto debugger

In [ ]:
bboxes = generate_clip_bboxes(toy, HEIGHT, WIDTH, STEP)
predict_clips(bboxes, toy)

## Predictions with ML Engine

In [ ]:
# read the blob contents from storage
# parse it as an image
# b64 encode the image
# make a prediction request
# save the results

# no. too much. Just run it local rn

In [ ]:
from PIL import Image, ImageDraw
from io import BytesIO
from oauth2client.client import GoogleCredentials
from googleapiclient import discovery
import base64

# Image Config
HEIGHT, WIDTH, DEPTH, STEP = 80, 80, 3, 25

# Model Config
PROJECT = 'reliable-realm-222318'
REGION = 'us-central1'
MODEL = 'satellite'
PROJECT_ID = 'projects/{}'.format(PROJECT)
MODEL_ID = PROJECT_ID + '/models/{}'.format(MODEL)

# Load image
image = Image.open('../../data/scenes/sfbay_1.png')

In [ ]:
def predict_cmle(bbox_gen, image):
    '''doc'''

    # Setup clients
    ml = discovery.build('ml', 'v1', cache_discovery=False)
    
    predictions = {}
    ship_count = 0
    total_count = 0
    
    for coords in bbox_gen:
        # crop clip
        clip = image.copy()
        clip = clip.crop(coords)
        
        # save to bytes
        image_bytes = BytesIO()
        clip.save(image_bytes, format='PNG')

        body = {'instances': {'image_bytes': {'b64': base64.b64encode(image_bytes.getvalue()).decode()}}}
        request = ml.projects().predict(name=MODEL_ID, body=body)
        response = request.execute()
        
#         print(coords, response)

        predictions[coords] = response

        for pred in response['predictions']:
            if pred['probabilities'][1] > 0.5:
                print('Found a ship at: {}!'.format(coords))
                print(response)
                ship_count += 1
        
#         input()
    return predictions

In [ ]:
bboxes = generate_clip_bboxes(image, HEIGHT, WIDTH, STEP)
predictions = predict_cmle(bboxes, image)

In [ ]:
for coords, pred in predictions.items():
    if pred['predictions'][0]['class'] == 'ship':
        print(coords, pred)

In [ ]:
# histogram of pos. probs
pos_probs = np.array([pred['predictions'][0]['probabilities'][1] for pred in predictions.values()])
neg_probs = np.array([pred['predictions'][0]['probabilities'][0] for pred in predictions.values()])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.hist(pos_probs, density=True, bins=1000);

In [ ]:
plt.figure(figsize=(12, 6))
plt.boxplot(pos_probs);

In [ ]:
import numpy as np

In [ ]:
pctile = np.percentile(pos_probs, 99)
pctile, pos_probs[pos_probs > pctile]

In [ ]:
pos_probs[pos_probs > 0.2]